# Workflow - Control


Install packages


In [1]:
!uv pip install -q \
    litellm==1.78.5 \
    python-dotenv==1.1.1 \
    pydantic==2.12.3

Import packages


In [ ]:
import json
import os
import random
import time
from typing import Literal, Tuple

import litellm  # type: ignore
from dotenv import load_dotenv  # type: ignore
from pydantic import BaseModel  # type: ignore

load_dotenv()

True

Define intent structured output


In [ ]:
class IntentClassification(BaseModel):
    intent: Literal["question", "request", "complaint"]
    confidence: float
    reasoning: str

In [ ]:
def route_based_on_intent(user_input: str) -> IntentClassification:
    messages = [
        {
            "role": "system",
            "content": (
                "Classify user input into one of three categories: "
                "question, request or complaint. Provide your reasoning and "
                "confidence level."
            ),
        },
        {"role": "user", "content": user_input},
    ]

    completion = litellm.completion(
        model="gemini/gemini-2.0-flash",
        messages=messages,
        response_format=IntentClassification,
    )

    content = completion.choices[0].message.content

    data = json.loads(content)

    return IntentClassification.model_validate(data)

In [ ]:
def answer_question(question: str) -> str:
    messages = [
        {
            "role": "user",
            "content": f"Answer this question in one sentence: {question}",
        },
    ]

    completion = litellm.completion(
        model="gemini/gemini-2.0-flash",
        messages=messages,
    )

    return completion.choices[0].message.content


def process_request(request: str) -> str:
    return f"Processing your request: {request}"


def handle_complaint(complaint: str) -> str:
    return (
        f"I understand your concern about {complaint}. Let me escalate this."
    )

In [ ]:
def handle_intent(intent: Literal["question", "request", "complaint"]):
    intent_handle = {
        "question": answer_question,
        "request": process_request,
        "complaint": handle_complaint,
    }

    return intent_handle[intent]

In [ ]:
user_inputs = [
    "What is generative AI?",
    "Please schedule a meeting for tomorrow",
    "I am not happy with service quality",
]

for user_input in user_inputs:
    intent_classification = route_based_on_intent(user_input)
    intent_handler = handle_intent(intent_classification.intent)
    handler_response = intent_handler(user_input)
    print(
        f"User input: {user_input}\n"
        f"User_intent: {intent_classification.intent}\n"
        f"Model confidence: {intent_classification.confidence}\n"
        f"Model reasoning: {intent_classification.reasoning}\n"
        f"Handler response: {handler_response}\n"
    )
    time.sleep(random.uniform(1.5, 3.0))

User input: What is generative AI?
User_intent: question
Model confidence: 0.95
Model reasoning: The user is asking a question about the definition of generative AI. This is a clear indication of an information-seeking query.
Handler response: Generative AI is a type of artificial intelligence that can create new content, such as text, images, audio, and video, based on the data it has been trained on.


User input: Please schedule a meeting for tomorrow
User_intent: request
Model confidence: 0.95
Model reasoning: The user is asking for an action to be performed (scheduling a meeting).
Handler response: Processing your request: Please schedule a meeting for tomorrow

User input: I am not happy with service quality
User_intent: complaint
Model confidence: 0.95
Model reasoning: The user expresses dissatisfaction with the service quality, indicating a complaint.
Handler response: I understand your concern about I am not happy with service quality. Let me escalate this.

